# Constants and Imports

## Imports

In [ ]:
from google.colab import drive

import os
import cv2
from google.colab.patches import cv2_imshow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.optimizers import Adam
from sklearn.utils import shuffle
from imutils import build_montages
import numpy as np
import argparse

## Constants

In [ ]:
IMG_DIMS = 28 #@param {type:"slider", min:16, max:256, step:32}
IMG_DIR = "/content/celeba/img_align_celeba/img_align_celeba" #@param {type:"string"}

OUTPUT_PATH = "/content/gdrive/MyDrive/Colab Files/output-celeba/" #@param {type:"string"}
#default:50
NUM_EPOCHS = 50 #@param {type:"slider", min:10, max:400, step:10}
#default128
BATCH_SIZE = 256 #@param {type:"slider", min:32, max:512, step:32}
INIT_LR = 02e-4 #@param {type:"number"}







# Prepare Kaggle-cli

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/') 

!pip install -q kaggle
!pip install -q kaggle-cli

!mkdir -p ~/.kaggle
!cp "/content/gdrive/MyDrive/Colab Files/Kaggle/kaggle.json" ~/.kaggle/
!cat ~/.kaggle/kaggle.json 
!chmod 600 ~/.kaggle/kaggle.json

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
{"username":"ihabtaleb","key":"59d8093446387a51abb5f7e82b9dd58e"}

# Download and Unzip The DataSet



In [ ]:
!kaggle datasets download -d jessicali9530/celeba-dataset

celeba-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip "/content/celeba-dataset.zip" -d "/content/celeba"

Archive:  /content/celeba-dataset.zip
replace /content/celeba/img_align_celeba/img_align_celeba/000001.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Load DataSet

In [ ]:
images=[]
for file_name in os.listdir(IMG_DIR):
  img = cv2.imread(os.path.join(IMG_DIR,file_name))
  if img.shape != (IMG_DIMS,IMG_DIMS):
    resized = cv2.resize(img,(IMG_DIMS,IMG_DIMS))
  images.append(resized)
trainImages=np.array(images)
cv2_imshow(trainImages[4300])
trainImages = (trainImages.astype("float") - 127.5) / 127.5
print(trainImages.shape)

(202599, 28, 28, 3)


# Generator and Discrimincator

## Generator


In [ ]:
def build_generator(dim, depth, channels=1, inputDim=100,
		outputDim=512):
		# initialize the model along with the input shape to be
		# "channels last" and the channels dimension itself
		model = Sequential()
		inputShape = (dim, dim, depth)
		chanDim = -1

		# first set of FC => RELU => BN layers
		model.add(Dense(input_dim=inputDim, units=outputDim))
		model.add(Activation("relu"))
		model.add(BatchNormalization())

		# second set of FC => RELU => BN layers, this time preparing
		# the number of FC nodes to be reshaped into a volume
		model.add(Dense(dim * dim * depth))
		model.add(Activation("relu"))
		model.add(BatchNormalization())

		# reshape the output of the previous layer set, upsample +
		# apply a transposed convolution, RELU, and BN
		model.add(Reshape(inputShape))
		model.add(Conv2DTranspose(32, (5, 5), strides=(2, 2),
			padding="same"))
		model.add(Activation("relu"))
		model.add(BatchNormalization(axis=chanDim))

		# apply another upsample and transposed convolution, but
		# this time output the TANH activation
		model.add(Conv2DTranspose(channels, (5, 5), strides=(2, 2),
			padding="same"))
		model.add(Activation("tanh"))

		# return the generator model
		return model





## Discriminator

In [ ]:
def build_discriminator(width, height, depth, alpha=0.2):
		# initialize the model along with the input shape to be
		# "channels last"
		model = Sequential()
		inputShape = (height, width, depth)

		# first set of CONV => RELU layers
		model.add(Conv2D(32, (5, 5), padding="same", strides=(2, 2),
			input_shape=inputShape))
		model.add(LeakyReLU(alpha=alpha))

		# second set of CONV => RELU layers
		model.add(Conv2D(64, (5, 5), padding="same", strides=(2, 2)))
		model.add(LeakyReLU(alpha=alpha))

		# first (and only) set of FC => RELU layers
		model.add(Flatten())
		model.add(Dense(512))
		model.add(LeakyReLU(alpha=alpha))

		# sigmoid layer outputting a single value
		model.add(Dense(1))
		model.add(Activation("sigmoid"))

		# return the discriminator model
		return model

# Build Models

In [ ]:
# build the generator
print("[INFO] building generator...")
gen = build_generator(7, 64, channels=3)
gen.summary()

# build the discriminator
print("[INFO] building discriminator...")
disc = build_discriminator(28, 28, 3)
discOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
disc.compile(loss="binary_crossentropy", optimizer=discOpt)
disc.summary()

# build the adversarial model by first setting the discriminator to
# *not* be trainable, then combine the generator and discriminator
# together
print("[INFO] building GAN...")
disc.trainable = False
ganInput = Input(shape=(100,))
ganOutput = disc(gen(ganInput))
gan = Model(ganInput, ganOutput)

# compile the GAN
ganOpt = Adam(lr=INIT_LR, beta_1=0.5, decay=INIT_LR / NUM_EPOCHS)
gan.compile(loss="binary_crossentropy", optimizer=discOpt)

[INFO] building generator...
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_12 (Dense)             (None, 512)               51712     
_________________________________________________________________
activation_15 (Activation)   (None, 512)               0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 512)               2048      
_________________________________________________________________
dense_13 (Dense)             (None, 3136)              1608768   
_________________________________________________________________
activation_16 (Activation)   (None, 3136)              0         
_________________________________________________________________
batch_normalization_10 (Batc (None, 3136)              12544     
_________________________________________________________________
reshape_3 (Reshape)      

# Train Models

In [ ]:
# randomly generate some benchmark noise so we can consistently
# visualize how the generative modeling is learning
print("[INFO] starting training...")
benchmarkNoise = np.random.uniform(-1, 1, size=(256, 100))

# loop over the epochs
for epoch in range(0, NUM_EPOCHS):
	# show epoch information and compute the number of batches per
	# epoch
	print("[INFO] starting epoch {} of {}...".format(epoch + 1,
		NUM_EPOCHS))
	batchesPerEpoch = int(trainImages.shape[0] / BATCH_SIZE)

	# loop over the batches
	for i in range(0, batchesPerEpoch):
		# initialize an (empty) output path
		p = None

		# select the next batch of images, then randomly generate
		# noise for the generator to predict on
		imageBatch = trainImages[i * BATCH_SIZE:(i + 1) * BATCH_SIZE]
		noise = np.random.uniform(-1, 1, size=(BATCH_SIZE, 100))

		# generate images using the noise + generator model
		genImages = gen.predict(noise, verbose=0)

		# concatenate the *actual* images and the *generated* images,
		# construct class labels for the discriminator, and shuffle
		# the data
		X = np.concatenate((imageBatch, genImages))
		y = ([1] * BATCH_SIZE) + ([0] * BATCH_SIZE)
		y = np.reshape(y, (-1,))
		(X, y) = shuffle(X, y)

		# train the discriminator on the data
		discLoss = disc.train_on_batch(X, y)

		# let's now train our generator via the adversarial model by
		# (1) generating random noise and (2) training the generator
		# with the discriminator weights frozen
		noise = np.random.uniform(-1, 1, (BATCH_SIZE, 100))
		fakeLabels = [1] * BATCH_SIZE
		fakeLabels = np.reshape(fakeLabels, (-1,))
		ganLoss = gan.train_on_batch(noise, fakeLabels)

		# check to see if this is the end of an epoch, and if so,
		# initialize the output path
		if i == batchesPerEpoch - 1:
			p = [OUTPUT_PATH, "epoch_{}_output.png".format(
				str(epoch + 1).zfill(4))]

		# otherwise, check to see if we should visualize the current
		# batch for the epoch
		else:
			# create more visualizations early in the training
			# process
			if epoch < 10 and i % 25 == 0:
				p = [OUTPUT_PATH, "epoch_{}_step_{}.png".format(
					str(epoch + 1).zfill(4), str(i).zfill(5))]

			# visualizations later in the training process are less
			# interesting
			elif epoch >= 10 and i % 100 == 0:
				p = [OUTPUT_PATH, "epoch_{}_step_{}.png".format(
					str(epoch + 1).zfill(4), str(i).zfill(5))]

		# check to see if we should visualize the output of the
		# generator model on our benchmark data
		if p is not None:
			# show loss information
			print("[INFO] Step {}_{}: discriminator_loss={:.6f}, "
				"adversarial_loss={:.6f}".format(epoch + 1, i,
					discLoss, ganLoss))

			# make predictions on the benchmark noise, scale it back
			# to the range [0, 255], and generate the montage
			images = gen.predict(benchmarkNoise)
			images = ((images * 127.5) + 127.5).astype("uint8")
			vis = build_montages(images, (28, 28), (16, 16))[0]

			# write the visualization to disk
			p = os.path.sep.join(p)
			cv2.imwrite(p, vis)

[INFO] starting training...
[INFO] starting epoch 1 of 50...
[INFO] Step 1_0: discriminator_loss=0.652204, adversarial_loss=0.528014
[INFO] Step 1_25: discriminator_loss=0.026715, adversarial_loss=0.002547
[INFO] Step 1_50: discriminator_loss=0.000486, adversarial_loss=0.000090
[INFO] Step 1_75: discriminator_loss=0.000063, adversarial_loss=0.000093
[INFO] Step 1_100: discriminator_loss=0.000033, adversarial_loss=0.000074
[INFO] Step 1_125: discriminator_loss=0.000030, adversarial_loss=0.000054
[INFO] Step 1_150: discriminator_loss=0.000021, adversarial_loss=0.000049
[INFO] Step 1_175: discriminator_loss=0.000043, adversarial_loss=0.000052
[INFO] Step 1_200: discriminator_loss=0.000056, adversarial_loss=0.000071
[INFO] Step 1_225: discriminator_loss=0.000117, adversarial_loss=0.000119
[INFO] Step 1_250: discriminator_loss=0.266756, adversarial_loss=1.129941
[INFO] Step 1_275: discriminator_loss=0.455306, adversarial_loss=1.540250
[INFO] Step 1_300: discriminator_loss=0.356712, adversar